In [6]:
from pyspark.sql import SparkSession

In [7]:
from pyspark.sql import SparkSession

# Spark session & Context
spark = (SparkSession
         .builder
         .master('local')
         .appName('kafka-streaming')
         # Add kafka package
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5")
         .getOrCreate())
sc = spark.sparkContext

In [19]:
# Read the message from the kafka stream
df = spark \
     .readStream \
     .format("kafka") \
     .option("kafka.bootstrap.servers", "kafka:9092") \
     .option("subscribe", "ingestion-topic") \
     .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38989)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:38989)

In [9]:
# Create a small temporary view for SparkSQL
df.createOrReplaceTempView("message")

In [10]:
# Write Out the message to the console of the environment
res = spark.sql("SELECT * FROM message")
res.writeStream.format("console") \
                .outputMode("append") \
                .start()

In [18]:
# Write the message back into Kafka in another topic that you are going to listen to with a local consumer
ds = df \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", "spark-output") \
    .option("checkpointLocation", "/tmp") \
    .start() \
    .awaitTermination()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38989)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:38989)